# test cuts.py and extinction.py

In [9]:
import os, sys
import glob 
import numpy as np
from astropy.table import Table, join

# --- plotting ---
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

from pfstarget import extinction as E
from pfstarget import cuts as Cuts

# extra functions

In [10]:
def _flux_to_mag(flux_nJy):
    flux_nJy = np.array(flux_nJy, dtype=np.float64)  
    mag = np.full_like(flux_nJy, np.nan) 
    
    mask = flux_nJy > 0  
    mag[mask] = -2.5 * np.log10(flux_nJy[mask] * 1e-32) - 48.6

    return mag

def quality_cuts_march2025(objects): 
    ''' quality cuts. Currently we only impose a magnitude depend cut on g-band
    magnitude error.
    '''
    # has g, r, i, y-band magnitudes 
    #cuts = (np.isfinite(objects['G_MAG']) & 
    #        np.isfinite(objects['R_MAG']) & 
    #        np.isfinite(objects['I_MAG']) & 
    #        np.isfinite(objects['Z_MAG']))

    # psf_flux_flag: psf_flux is required by the observatory
    cuts = ((~objects['G_PSF_FLAG']) & 
             (~objects['R_PSF_FLAG']) & 
             (~objects['I_PSF_FLAG']) &
             (~objects['Z_PSF_FLAG']))
    
    print('psf_flux_flag cut remove:', len(objects) - np.sum(cuts), 'objects')

    # g-band magnitude error cut (NOTE: G_MAG here is dust corrected, for march2025, G_MAG used for this cut is not dust corrected!)
    cuts_sn = (objects['G_ERR'] < objects['G_MAG_0'] * 0.05 - 1.1) 
    cuts &= (objects['G_ERR'] < objects['G_MAG_0'] * 0.05 - 1.1) 
    print('g-band magnitude error remove:', len(objects) - np.sum(cuts_sn), 'objects')

    # not skipped by deblender (this is a very very small fraction of objects)
    #cuts &= (~objects['DEBLEND_SKIPPED'])

    # low surface brightness object cut (ref. Li Xiangchong et al. 2022, Table
    # 2; also Issue #10) 
    cuts_low_sb = ((objects['I_APFLUX10_MAG'] <= 25.5) & 
             (~objects['I_APFLUX10_FLAG']))
    cuts &= ((objects['I_APFLUX10_MAG'] <= 25.5) & 
             (~objects['I_APFLUX10_FLAG']))
    print('low surface brightness cut remove:', len(objects) - np.sum(cuts_low_sb), 'objects')

    # cut on extreme colors (these values are preliminary --- revisit this in detail)
    #cuts &= ((objects['G_MAG'] - objects['R_MAG'] > -1) & 
    #         (objects['I_MAG'] - objects['Z_MAG'] > -1))
    return cuts

# NOTE: default values here are set to the values used in march2025 targets
def isCosmology_march2025(objects, star_galaxy_cut=-0.08, magnitude_cut=22.5,
                g_r_cut=0.35, color_slope=2.0, color_yint=0.55):
    ''' Select targets for the PFS Cosmology Survey for march2025 

    args: 
        objects: HSC objects 

    kwargs: 
        star_galaxy_cut: cut for the star-galaxy separation. (Default: -0.15) 

    
    references:
    ----------
    * DESI elg target selection https://github.com/desihub/desitarget/blob/main/py/desitarget/cuts.py#L646

    '''
    # mask
    # is_mask = masking(objects) 

    # quality cuts
    is_quality = quality_cuts_march2025(objects)
    print('quality cut remove:', len(objects) - np.sum(is_quality), 'objects')

    # star-galaxy separation
    is_galaxy = Cuts.star_galaxy(objects, cut=star_galaxy_cut)
    print('star-galaxy cut remove:', len(objects) - np.sum(is_galaxy), 'objects')

    # color cut
    is_color = Cuts.color_cut(objects, magnitude_cut=magnitude_cut, 
                         g_r_cut=g_r_cut, color_slope=color_slope,
                         color_yint=color_yint) 
    print('after color cut remove:', len(objects) - np.sum(is_color), 'objects')

    return is_quality & is_galaxy & is_color 

In [18]:
import sys
from argparse import ArgumentParser

# Simulate command-line arguments
sys.argv = [
    "test_cuts_extinction.ipynb",  # Script name (ignored by ArgumentParser)
    "/Users/jing/My_Work/Ongoing/PFS_CO_Repo/pfs_co_fa/data_proc/s23b_wide/tracts/",          # First argument: tractsdir
    "./",               # Second argument: dest
    "--dust", "desi"              # Optional argument: dust model
]

ap = ArgumentParser(description='Generate PFS targets from HSC tract files')
ap.add_argument("tractsdir",
                help="tract file or root directory with tract files")
ap.add_argument("dest",
                help="Output target selection directory")
ap.add_argument("--dust", type=str,
                help='galactic extinction dust model [defaults to desi dust map]',
                default='desi')
ns = ap.parse_args()

infiles = [] 
if os.path.isfile(ns.tractsdir): infiles = glob.glob(ns.tractsdir) 
elif os.path.isdir(ns.tractsdir): infiles = glob.glob('%s/*' % ns.tractsdir) 
else: raise ValueError("no tract files found") 

if len(infiles) == 0:
    raise ValueError("no tract files found") 
    sys.exit(1)

# output file name 
if os.path.isfile(ns.dest): 
    fout = ns.dest
elif os.path.isdir(ns.dest): 
    fout = os.path.join(ns.dest, f'pfs_target.dust_{ns.dust}.fits') 
else: 
    raise ValueError('specify output directory or filename')  

# loop through tract files 
# and select PFS cosmology targets
targets = [] 
for infile in infiles: 
    # read tract file 
    tract = Table.read(infile, format='fits')
    print(f"Processing {infile}...")

    # rename column names -- TODO: make the column names (in Jingjing's python script) to be consistent with SQL script 
    tract.rename_column('i_cmodel_flux_meas', 'i_meas_cmodel_flux')
    tract.rename_column('i_cmodel_mag_meas', 'i_meas_cmodel_mag')
    tract.rename_column('i_cmodel_flag_meas', 'i_meas_cmodel_flag')
    tract.rename_column('i_psf_flux_meas', 'i_meas_psf_flux')
    tract.rename_column('i_psf_mag_meas', 'i_meas_psf_mag')
    tract.rename_column('i_psf_flag_meas', 'i_meas_psf_flag')

    # add bright star mask and SFD98 dust attenuation
    fn = os.path.basename(infile)
    fn_bs_photoz = fn.split('_')[-1]
    dir_bs_photoz = "/Users/jing/My_Work/Ongoing/PFS_CO_Repo/pfs_co_fa/sql/database/s23b-bsmask/tracts/"
    bs_photoz = Table.read(dir_bs_photoz + fn_bs_photoz, format='fits')
    
    tract['a_g'] = bs_photoz['a_g']
    tract['a_r'] = bs_photoz['a_r']
    tract['a_i'] = bs_photoz['a_i']
    tract['a_z'] = bs_photoz['a_z']
    tract['a_y'] = bs_photoz['a_y']
    tract['i_mask_brightstar_halo'] = bs_photoz['i_mask_brightstar_halo']
    tract['i_mask_brightstar_blooming'] = bs_photoz['i_mask_brightstar_blooming']
    tract['i_mask_brightstar_ghost'] = bs_photoz['i_mask_brightstar_ghost']
    tract['i_apertureflux_10_mag'] = _flux_to_mag(tract['i_apertureflux_10_flux'])

    # preprocess tract file (using specified galactic extinction dust model) 
    _hsc = Cuts._prepare_hsc(tract, dust_extinction=ns.dust, zeropoint=False)
    print(f"Number of HSC sources in tract {fn}: {len(_hsc)}")

    # apply PFS S25A march run target selection
    # NOTE: magerr cut is different from what was used in the march2025 (see above)
    is_pfscosmo = isCosmology_march2025(_hsc)
    print(f"Number of PFS cosmology targets of S25A march run in tract {fn}: {np.sum(is_pfscosmo)}")

    # apply PFS cosmology target selection 
    #is_pfscosmo = Cuts.isCosmology(_hsc)
    #print(f"Number of PFS cosmology targets in tract {fn}: {np.sum(is_pfscosmo)}")
    
    # targets 
    targs = _hsc[is_pfscosmo]
    targets.append(targs) 

# combine all targets 
targets = Table(np.concatenate(targets)) 

# write to file 
targets.write(fout, format='fits', overwrite=True)
print(f"Targets written to {fout}")  

Processing /Users/jing/My_Work/Ongoing/PFS_CO_Repo/pfs_co_fa/data_proc/s23b_wide/tracts/s23b_gal_9563.fits...
Number of HSC sources in tract s23b_gal_9563.fits: 264458
psf_flux_flag cut remove: 3 objects
g-band magnitude error remove: 28829 objects
low surface brightness cut remove: 15318 objects
quality cut remove: 40654 objects
star-galaxy cut remove: 28697 objects
magnitude cut removed: 159982
color cut removed: 160520
after color cut remove: 222300 objects
Number of PFS cosmology targets of S25A march run in tract s23b_gal_9563.fits: 40615
Targets written to ./pfs_target.dust_desi.fits


In [13]:
_hsc[is_pfscosmo]

array([(3406823584524403124, 139.76819, 0.28224435, 23.837194, 23.613436, 23.505573, 23.40458 , 23.617914, 0.03718814, 0.05039589, 0.04408505, 0.09005562, 0.17411482, 23.58529 , 24.44858 , False, False, 0, 0, 0, False, False, False, False, False, 24.690645, False, 23.99536 , 23.724543, 23.58529 , 23.466234, 23.670391),
       (3406823584524403191, 139.84087, 0.28361833, 23.944424, 24.138119, 23.632515, 23.692919, 23.428984, 0.05070275, 0.07916424, 0.05235006, 0.09991338, 0.09732133, 23.712233, 24.555225, False, False, 0, 0, 0, False, False, False, False, False, 24.8523  , False, 24.102589, 24.249226, 23.712233, 23.754574, 23.48146 ),
       (3406823584524403214, 139.81224, 0.28427997, 24.415895, 24.188501, 23.915565, 23.34105 , 23.061573, 0.04708118, 0.06193825, 0.03752062, 0.07052004, 0.05024991, 23.995283, 24.269022, False, False, 0, 0, 0, False, False, False, False, False, 24.527685, False, 24.57406 , 24.299608, 23.995283, 23.402704, 23.11405 ),
       ...,
       (34070830692685844

In [14]:
len(targets), targets.keys(), targets[0:5]

(40615,
 ['OBJID',
  'RA',
  'DEC',
  'G_MAG',
  'R_MAG',
  'I_MAG',
  'Z_MAG',
  'Y_MAG',
  'G_ERR',
  'R_ERR',
  'I_ERR',
  'Z_ERR',
  'Y_ERR',
  'I_MEAS_CMODEL_MAG',
  'I_MEAS_PSF_MAG',
  'I_MEAS_CMODEL_FLAG',
  'I_MEAS_PSF_FLAG',
  'I_MASK_HALO',
  'I_MASK_GHOST',
  'I_MASK_BLOOMING',
  'G_PSF_FLAG',
  'R_PSF_FLAG',
  'I_PSF_FLAG',
  'Z_PSF_FLAG',
  'DEBLEND_SKIPPED',
  'I_APFLUX10_MAG',
  'I_APFLUX10_FLAG',
  'G_MAG_0',
  'R_MAG_0',
  'I_MAG_0',
  'Z_MAG_0',
  'Y_MAG_0'],
 <Table length=5>
        OBJID            RA       DEC     ...  I_MAG_0   Z_MAG_0   Y_MAG_0 
        int64         float32   float32   ...  float32   float32   float32 
 ------------------- --------- ---------- ... --------- --------- ---------
 3406823584524403124 139.76819 0.28224435 ...  23.58529 23.466234 23.670391
 3406823584524403191 139.84087 0.28361833 ... 23.712233 23.754574  23.48146
 3406823584524403214 139.81224 0.28427997 ... 23.995283 23.402704  23.11405
 3406823584524403262 139.83554 0.28544325 ..

In [15]:
tract.colnames

['object_id',
 'ra',
 'dec',
 'tract',
 'patch',
 'g_cmodel_flux',
 'g_cmodel_flux_err',
 'g_cmodel_mag',
 'g_cmodel_mag_err',
 'g_psf_flag',
 'g_psf_flux',
 'g_psf_flux_err',
 'r_cmodel_flux',
 'r_cmodel_flux_err',
 'r_cmodel_mag',
 'r_cmodel_mag_err',
 'r_psf_flag',
 'r_psf_flux',
 'r_psf_flux_err',
 'i_cmodel_flux',
 'i_cmodel_flux_err',
 'i_cmodel_mag',
 'i_cmodel_mag_err',
 'i_psf_flag',
 'i_psf_flux',
 'i_psf_flux_err',
 'z_cmodel_flux',
 'z_cmodel_flux_err',
 'z_cmodel_mag',
 'z_cmodel_mag_err',
 'z_psf_flag',
 'z_psf_flux',
 'z_psf_flux_err',
 'y_cmodel_flux',
 'y_cmodel_flux_err',
 'y_cmodel_mag',
 'y_cmodel_mag_err',
 'y_psf_flag',
 'y_psf_flux',
 'y_psf_flux_err',
 'i_apertureflux_10_flux',
 'i_apertureflux_10_flux_err',
 'i_apertureflux_10_flag',
 'i_meas_cmodel_flux',
 'i_meas_cmodel_mag',
 'i_meas_cmodel_flag',
 'i_meas_psf_flux',
 'i_meas_psf_mag',
 'i_meas_psf_flag',
 'i_extendedness_value',
 'i_extendedness_flag',
 'deblend_skipped',
 'a_g',
 'a_r',
 'a_i',
 'a_z',
 'a

In [16]:
bs_photoz.colnames

['object_id',
 'object_id_isnull',
 'ra',
 'ra_isnull',
 'dec',
 'dec_isnull',
 'tract',
 'tract_isnull',
 'patch',
 'patch_isnull',
 'a_g',
 'a_g_isnull',
 'a_r',
 'a_r_isnull',
 'a_i',
 'a_i_isnull',
 'a_z',
 'a_z_isnull',
 'a_y',
 'a_y_isnull',
 'g_mask_brightstar_halo',
 'g_mask_brightstar_halo_isnull',
 'g_mask_brightstar_dip',
 'g_mask_brightstar_dip_isnull',
 'g_mask_brightstar_ghost',
 'g_mask_brightstar_ghost_isnull',
 'g_mask_brightstar_blooming',
 'g_mask_brightstar_blooming_isnull',
 'g_mask_brightstar_ghost12',
 'g_mask_brightstar_ghost12_isnull',
 'g_mask_brightstar_ghost15',
 'g_mask_brightstar_ghost15_isnull',
 'r_mask_brightstar_halo',
 'r_mask_brightstar_halo_isnull',
 'r_mask_brightstar_dip',
 'r_mask_brightstar_dip_isnull',
 'r_mask_brightstar_ghost',
 'r_mask_brightstar_ghost_isnull',
 'r_mask_brightstar_blooming',
 'r_mask_brightstar_blooming_isnull',
 'r_mask_brightstar_ghost12',
 'r_mask_brightstar_ghost12_isnull',
 'r_mask_brightstar_ghost15',
 'r_mask_brightst